## BMDS benchmarks tooling

Last updated: *2023-08-08*

The Python BMDS package has an benchmarks package to make it easier to store results from multiple operating systems and versions of the software. This tooling is used as part of our software development lifecycle to better understand the final impact of changes to the software.  This notebook demonstrates using the benchmarks package to store and retrieve results.

## Executing an session in batch mode

A general approach taken would be execute the same suite of datasets across multiple versions of the software and/or environments. This will demonstrate using the back mode in this form.

In [ ]:
import json
from pathlib import Path

data = json.loads(Path("./data/dich_30.json").read_text())
print(len(data))

In [ ]:
import bmds
from bmds import constants
from bmds.bmds3.constants import PriorClass
from bmds.bmds3.types.dichotomous import DichotomousRiskType
from bmds.benchmarks import BmdsSessionWrapper, TblSession

def build_dichotomous_dataset(ds_dict: dict) -> bmds.datasets.DichotomousDataset:
    if ds_dict["dtype"] == constants.Dtype.DICHOTOMOUS:
        return bmds.datasets.DichotomousDataset(
            ds_dict["doses"],
            ds_dict["ns"],
            ds_dict["incidences"],
            **ds_dict["metadata"],
        )
    else:
        raise ValueError("Invalid dtype")


def build_dich_session(ds, version):
    def add_model(sess, Model, base: tuple, additions=None):
        settings = {
            "id": base[0],
            "priors": PriorClass.frequentist_restricted,
            "bmr_type": base[1],
            "bmr": base[2],
        }
        if additions is not None:
            settings.update(additions)
        sess.add_model(Model, settings, base[0])

    option_sets = [
        (r"10% Extra Risk", DichotomousRiskType.ExtraRisk, 0.1),
        (r"10% Added Risk", DichotomousRiskType.AddedRisk, 0.1),
    ]
    sessions = []
    for index, option_set in enumerate(option_sets):
        sess = BmdsSessionWrapper(
            bmds_version=version, dataset=ds, session_name=f"session_{ds.metadata.name}_{index}"
        )

        # fmt: off
        # add_model(sess, constants.M_DichotomousHill, option_set)
        # add_model(sess, constants.M_DichotomousHill, option_set, {"priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_Gamma, option_set)
        # add_model(sess, constants.M_Gamma, option_set, {"priors": PriorClass.frequentist_unrestricted})
        add_model(sess, constants.M_Logistic, option_set, {"priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_LogLogistic, option_set)
        add_model(sess, constants.M_LogLogistic, option_set, {"priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_LogProbit, option_set)
        # add_model(sess, constants.M_LogProbit, option_set, {"priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_Multistage, option_set, {"degree": 1})
        # add_model(sess, constants.M_Multistage, option_set, {"degree": 1, "priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_Multistage, option_set, {"degree": 2})
        # add_model(sess, constants.M_Multistage, option_set, {"degree": 2, "priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_Multistage, option_set, {"degree": 3})
        # add_model(sess, constants.M_Multistage, option_set, {"degree": 3, "priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_Probit, option_set, {"priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_QuantalLinear, option_set, {"priors": PriorClass.frequentist_unrestricted})
        # add_model(sess, constants.M_Weibull, option_set)
        # add_model(sess, constants.M_Weibull, option_set, {"priors": PriorClass.frequentist_unrestricted})
        # fmt: on
        sessions.append(sess)

    return sessions


Execute the analysis:

In [ ]:
def run_dichotomous_frequentist(ds_dicts, version):
    dich_ds = [build_dichotomous_dataset(ds_dict) for ds_dict in ds_dicts]
    sessions = []
    for ds in dich_ds[:5]:
        sessions.extend(build_dich_session(ds, version))
    return BmdsSessionWrapper.bulk_execute_and_recommend(sessions)

sessions = run_dichotomous_frequentist(data, constants.Version.BMDS330)

Format data for adding to the database:

Write outputs to the database:

In [ ]:
def insert_results(sessions):
    results = [TblSession.from_bmds(session.session) for session in sessions]
    with SQLSession() as conn:
        for result in results:
            conn.add(result)
        conn.commit()

insert_results(sessions)

## Querying the benchmarks database

After results have been executed, we can query the database to fetch results.

In [ ]:
from bmds.benchmarks import TblModelResultScalar, TblModel, TblSession, SQLSession
from sqlmodel import select
from sqlalchemy.orm import contains_eager

statement = (
    select(TblModelResultScalar)
    .join(TblModelResultScalar.model)
    .join(TblModel.session)
    .options(contains_eager(TblModelResultScalar.model, TblModel.session))
)
with SQLSession() as session:
    results = session.exec(statement).unique().all()

In [ ]:
import pandas as pd

df = TblModelResultScalar.get_df(results)

In [ ]:
len(df)


In [ ]:
import plotly.express as px

fig = px.scatter(df, y="value", x="attribute", color="model_name")
fig.show()